# Project

## Name: Hrishikesh Dinkar Kanade


References:
1. https://stackoverflow.com/questions/2669059/how-to-sort-alpha-numeric-set-in-python
2. https://wangyy395.medium.com/implement-a-trie-in-python-e8dd5c5fde3a
3. https://www.geeksforgeeks.org/compressed-tries/#
4. Algorithm Design and Applications by Goodrich and Tamassia
5. ChatGPT (For clarifying concepts)
6. CS 600 Lecture slides
7. https://en.wikipedia.org/wiki/Trie
8. https://www.crummy.com/software/BeautifulSoup/bs4/doc/


### Import libraries

In [1]:
import numpy as np
import os
from bs4 import BeautifulSoup as bs
import re
from collections import defaultdict
from tqdm import tqdm

### Read text from the files

*Update the file path when running on your system*

In [2]:
input_folder = "C:/Users/kanad/Documents/CS 600/Project/HTML_files" # Location of HTML files

In [3]:
text_dictionary = {} # Dictionary format: {Webpage: text}

for fname in os.listdir(input_folder):   #iterate over files in directory
    fpath = os.path.join(input_folder, fname)
    print(f"Reading: {fpath}")
    with open(fpath, 'r', encoding='utf-8') as f:
        bs_obj = bs(f, 'html.parser')
        txt = bs_obj.get_text()
        text_dictionary[fpath] = txt 

Reading: C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html
Reading: C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html
Reading: C:/Users/kanad/Documents/CS 600/Project/HTML_files\Brown_bear_ Wikipedia.html
Reading: C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html
Reading: C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html
Reading: C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html


## Tokenize and filter data

### Tokenize

In [4]:
tokenized_webpages_dict ={}

for webpage, txt in text_dictionary.items():
    tokenized = re.findall(r'\b[a-zA-Z]+\b', txt.lower())
    tokenized_webpages_dict[webpage] = tokenized

### Filter stopwords.

Stopwords imported from NLTK and filtered from the input data

In [5]:
from nltk.corpus import stopwords
filter_words = set(stopwords.words('english'))

filtered_webpages_dict = {}
vocab = set()

for webpage, tokens in tokenized_webpages_dict.items():
    filtered = [word for word in tokens if word not in filter_words]
    filtered_webpages_dict[webpage] = filtered
    vocab.update(filtered)

In [6]:
print(f"Number of words in vocabulary: {len(vocab)}")

Number of words in vocabulary: 9877


## Create inverted index

The inverted index contains webpages the word appears in and the number of times it appears

In [138]:
inverted_indices = defaultdict(set)

for webpage, word in filtered_webpages_dict.items():
    seen_words=[] # maintain a seen words list to avoid repeating word count for same word
    for i in word:
        if i not in seen_words:
            inverted_indices[i].add(webpage+' '+str(word.count(i))+' instances') ## Add the webpage and count of instances for word
            seen_words.append(i)

In [91]:
len(inverted_indices)

9877

### Implement the Trie

In [205]:
class Comp_Trie_Node:
    def __init__(self):
        self.child_nodes = {}
        self.is_last = False
    
    def print_node(self, prefix='', indent=''):
        terminal_marker = '*' if self.is_last else ''
        if prefix:
            print(f"{indent}{prefix}{terminal_marker}")
        for edge_label, child_node in self.child_nodes.items():
            child_node.print_node(edge_label, indent + '  ')

class Compressed_Trie:
    def __init__(self):
        self.root_node = Comp_Trie_Node()
        
    def longest_common_prefix(self, string1, string2):
        minimum_length = min(len(string1), len(string2))
        i = 0
        while i < minimum_length and string1[i] == string2[i]:
            i += 1
        return string1[:i]
    
    def print_trie(self):
        print("Compressed Trie Structure:")
        self.root_node.print_node()
        
    def insert(self, word):
        node = self.root_node
        word_subword = word
        while True:
            if not word_subword:
                node.is_last = True
                return
            for prefix_label, child_item in node.child_nodes.items():
                lc_prefix = self.longest_common_prefix(prefix_label, word_subword)
                if lc_prefix:
                    if lc_prefix == prefix_label:
                        node = child_item
                        word_subword = word_subword[len(lc_prefix):]
                        break
                    else: # split edge if partial match is found
                        existing_child = child_item 
                        new_node = Comp_Trie_Node() # create a node for prefixes
                        remaining_label = prefix_label[len(lc_prefix):]
                        if remaining_label:
                            new_node.child_nodes[remaining_label] = existing_child
                            new_node.is_last = False
                        else: # new node takes existing_child's children
                            new_node.child_nodes = existing_child.child_nodes
                            new_node.is_last = existing_child.is_last

                        node.child_nodes.pop(prefix_label) ## update children of current node
                        node.child_nodes[lc_prefix] = new_node

                        remaining_word = word_subword[len(lc_prefix):] # add remainder as child
                        if remaining_word:
                            new_leaf_node = Comp_Trie_Node()
                            new_leaf_node.is_last = True
                            new_node.child_nodes[remaining_word] = new_leaf_node
                        else:
                            new_node.is_last = True
                        return
            else: # no prefix found so add word as new child
                new_leaf_node = Comp_Trie_Node()
                new_leaf_node.is_last = True
                node.child_nodes[word_subword] = new_leaf_node
                return

    def search_function(self, word): ## funstion for searching the trie
        node = self.root_node
        word_subword = word
        
        while True:
            if not word_subword:
                return node.is_last
            found_prefix = False
            for prefix_label, child in node.child_nodes.items():
                lc_prefix = self.longest_common_prefix(prefix_label, word_subword)
                if lc_prefix == prefix_label:
                    if lc_prefix == word_subword:
                        return child.is_last
                    else:
                        node = child
                        word_subword = word_subword[len(lc_prefix):]
                        found_prefix = True
                        break
                elif lc_prefix == word_subword:
                    return False
            if not found_prefix:
                return False

### Create the Trie

In [206]:
from tqdm import tqdm
vocab_trie = Compressed_Trie()

for item in tqdm(vocab):
    vocab_trie.insert(item)

100%|███████████████████████████████████████████████████████████████████████████| 9877/9877 [00:00<00:00, 59503.71it/s]


In [220]:
vocab_trie.print_trie()

Compressed Trie Structure:
  c*
    h*
      e
        rry*
        z*
        w
          s*
          ing*
        n*
          gd
            u*
            ong*
        m
          i
            cal*
              s*
            stry*
          osignals*
        e
          k*
            s*
          tah*
        st*
          in*
        vron*
      a
        n*
          g*
            ing*
            e*
              taxa*
              s*
                upload*
          nel*
            s*
          ce*
            s*
          dra*
        s
          e*
            d*
          ing*
        p
          man*
          ter*
          ultepec*
          a
            lmalania*
            rral*
        r
          l
            ton*
            es*
            otte*
          acter*
            s*
            i
              bus*
              z
                ed*
                ation*
              s
                ed*
                tics*
          ismatic*
          c

          ard*
          d*
          hard*
            t*
        lin*
        r
          ies*
          y*
        g
          man*
            n*
          er*
        e
          zkin*
          czky*
        a*
          rdinelli*
        keley*
      doya*
      n*
        e
          volent*
          fi
            t*
              s*
            cial*
        gal*
          ensis*
        d*
        veniste*
        n*
          y*
          ett*
            ii*
        jamin*
      l
        l*
          y*
          ied*
          emain*
          ows*
        ant*
        i
          n*
          e
            f*
            ve*
              d*
          kov*
        uga*
          s*
        o
          ng*
            s*
          ved*
        t*
      h
        ind*
        eco*
        avio
          r*
            al*
            s*
          ur*
            al*
            s*
        rend*
      e*
        hive*
          s*
        ch*
          am*
        r*
    

      a*
        sbt*
        ng*
      e
        ttmann*
        s*
      g
        ging*
        e*
        h*
          es*
      o*
      ck
        leberries*
        ins*
      l
        bert*
        l*
      dson*
    y
      p
        othes
          i
            zed*
            s*
          es*
        er
          phagia*
          carnivore*
      ena*
        s*
      brid*
        s*
        ization*
          s*
      strix*
      a
        en
          inae*
          a*
        des*
      d
        er*
        r
          urga*
          ictis*
    e
      r
        b
          al*
          ivor
            es*
              endangered*
              herbivorous*
            y*
            ous*
        ald*
          ry*
        d
          s*
          ing*
          er*
            s*
        r
          in*
          er
            o*
            a*
        tfordshire*
        ma
          phroditus*
          n*
        e
          dity*
          related*
     

          i
            on*
            ng*
            ve*
            b
              le*
              ility*
          s*
          ed*
        it*
          s*
          al*
        ging*
      m
        ensions*
        orphi
          c*
          sm*
        inuti
          ve*
          on*
      urnal*
      r
        ofilaria*
        ect*
          i
            ons*
            ve*
          ly*
          or*
        t*
      lemma*
      c
        t
          ate*
          ionar
            ies*
            y*
        k*
          man*
          inson*
        hromats*
      n
        g*
          zhen*
        ets*
        ogale*
      ff
        er*
          en
            ces*
            t*
              ly*
              i
                is*
                ate*
                  d*
        icult*
    r
      o
        p*
          s*
          p
            ed*
            ing*
              s*
        wn
          ing*
          ed*
      a
        g*
          

      m*
        ber*
          ing*
        pkin*
        i*
      n*
        g
          s*
          es*
        de*
        n*
      p
        aster*
        u
          s*
          lella*
      dlow*
      t
        r
          i
            s*
            ctis*
            nae*
          a*
            vus*
            eximia*
          eol
            ina*
            a*
          ogale*
        eolus*
      igi*
      s
        cious*
        ter*
      ke*
      ring*
      o*
        gale*
      c
        k*
        y*
    win*
    s*
    vs*
    b*
    lc*
    td*
    h
      o*
      am*
  o
    t
      ter*
        s*
      oc
        olobus*
        yon*
      hers*
      ari
        i
          nae*
          dae*
        a*
    o*
    r
      phan*
      egon*
      d
        i
          z*
          n
            es*
            aries*
        er*
      i
        en
          ta
            tion*
            lis*
          sictis*
        gin*
          a
            

        ease*
          d*
        i
          gion*
          a
            nt*
            bly*
          es*
        a
          t
            ed*
            i
              on*
                s*
                  hip*
                    s*
              ve*
                ly*
                s*
          xed*
        ocate*
      b
        a*
        ecca*
        irth*
        uilding*
      d*
        head*
        irect*
          s*
        uc
          e*
            s*
            d*
          tion*
            s*
        wood*
        tenbacher*
        efined*
        d
          y*
          ish*
      eder*
      t
        ired*
        ention*
        ain*
          s*
          ed*
        r
          act
            able*
            ile*
          eat*
            ing*
          ieved*
          ospective*
        urn*
          ed*
        old*
      v
        e
          al*
            s*
            ed*
          nge*
          rse*
            d*
        i
  

      x*
        es*
        ual*
          ly*
      e*
        ing*
        ds*
        k*
        blatt*
        n*
        m*
          ed*
      d
        ges*
        entary*
        ucing*
      m
        i*
          striatus*
          torquata*
        ant
          ically*
          or*
        e
          n*
          carpifolia*
      que
        ster*
        nc
          ing*
          e*
            s*
            d*
        ira*
      p
        tember*
        arat
          ion*
          e*
            d*
      bastien*
      i
        zes*
        densticker*
    k
      y*
      e
        tches*
        leton*
          bear*
      i
        ll
          s*
          ful*
        n*
          s*
      u
        nk*
          s*
        ll*
          red*
          s*
      opelogale*
      ates*
    l
      uggishness*
      o
        w*
          ly*
          s*
          e
            r*
            d*
        v
          enia*
          ak*
            ia*
    

        graphic*
          al*
            ly*
          furth*
        chronology*
        ffroy*
          i*
        bios*
        logical*
        metr
          y*
          ic*
        rg
          ia*
          e*
      bauer*
      t*
        ting*
        s*
      stati
        on*
        ng*
      d*
      ert*
      cco*
    r*
      e
        at*
          ly*
          e
            r*
            st*
        w*
        e
          ting*
          k*
          n*
            land*
            peace*
            ing*
            wood*
          ce*
        g
          arious*
          ory*
        y*
      o
        ves*
        w*
          n*
            ups*
          ing*
          s*
          l*
            s*
            ing*
          th*
        ss*
        en
          ewald*
          landicus*
        u
          p*
            s*
            ing*
          nd*
          se*
        o
          ves*
          m*
      a
        n
          ny*
          t*
   

            a*
        ur
          e*
            serve*
            lle*
            conservation*
          kunde*
          a
            l*
              ly*
              ist*
                ic*
            e*
        han*
      emorhedus*
      n
        ook*
        iwadekar*
        dinia*
      u
        mov*
        ghton*
        ka*
      hmo*
      v
        igation*
        ajo*
        es*
      id*
        u*
      ked*
      chtigall*
      g
        arhole*
        y*
    u
      t*
        s*
        ri
          ous*
          ent*
            s*
          tio
            n*
              al*
            us*
      isance*
      r
        ture*
        s
          ing*
          e*
            s*
      dipes*
      m
        ber*
          s*
        e
          n*
          rous*
      liajuk*
      clear*
    yctereutes*
    cbi*
    b*
      c*
      s*
      n*
    p
      s*
      r*
    j*
    g*
      uyen*
  t
    r
      i
        es*
        m*
        vi

        t
          e*
            side*
            b
              eam*
              ark*
          ish*
      yte*
      e
        never*
        at*
        ther*
        re
          ver*
          as*
      b*
      o
        se*
        le*
      al
        e*
          rs*
          s*
        ing*
      c*
    r
      o
        e*
        te*
      e
        ath*
        stl
          ing*
          er*
      i
        nkled*
        t
          ings*
          ten*
          er*
        st*
    y*
      oming*
      vern*
    w
      w*
      f*
        china*
    jhg*
    sb*
    m
      sbga*
      on*
    u*
      nn*
      rsig*
  y
    le*
    np*
    u*
      ri*
      shania*
      kon*
      n*
        nan*
      liy*
      e*
        yue*
        n*
      an*
    e
      ung*
      t*
        i*
      l
        low*
          stone*
            park*
          ish*
        e*
      soensis*
      ar*
        s*
        book*
        l
          y*
          ing*
   

## Search and ranking

### Function for search and ranking

In [190]:
def multiple_search_webpages(searchword, vocab_trie, inverted_indices,filter_words):
    string_list = re.findall(r'\b[a-zA-Z]+\b', searchword.lower()) # preprocess
    filtered_string_list = [word for word in string_list if word not in filter_words]
    print(f"input: {searchword} \n")
    
    results_list = []
    intersected_pages = None  # store common webpages
    
    for searchword in filtered_string_list:
        if vocab_trie.search_function(searchword):  # check if word is in trie
            pages = inverted_indices.get(searchword, [])  # set the webpages from word with inverted index
            sorted_pages = sorted(pages, key=lambda string_in: int(string_in.split()[-2]), reverse=True)  # sort by no. of occurences
            print(f"'{searchword}' found in these pages: \n")
            for i in sorted_pages:  # Print the ranked results
                print(i)
            print('\n')
            if len(filtered_string_list)>1: # check for intersection of webpages, this will give common terms
                trimmed_pages = {page.split("html")[0] + "html" for page in sorted_pages}
                if intersected_pages is None:
                    intersected_pages = trimmed_pages
                else:
                    intersected_pages &= trimmed_pages 
        else:
            print(f"Word '{searchword}' is not in vocabulary.")  # word is not in trie
    
    if len(filtered_string_list)>1: # print common webpages if any
        if intersected_pages:
            print("\nIntersection of results for all words:\n")
            for page in intersected_pages:
                print(page)
        else:
            print("\nNo common pages found for all words.")

### Sample searches

In [191]:
searchword='pandas'
multiple_search_webpages(searchword, vocab_trie, inverted_indices,filter_words)

input: pandas 

'pandas' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 123 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 45 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 12 instances




In [204]:
searchword='potato'
multiple_search_webpages(searchword, vocab_trie, inverted_indices,filter_words)

input: potato 

Word 'potato' is not in vocabulary.


In [211]:
searchword='extinction of pandas'
multiple_search_webpages(searchword, vocab_trie, inverted_indices,filter_words)

input: extinction of pandas 

'extinction' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Brown_bear_ Wikipedia.html 5 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 1 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 1 instances


'pandas' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 123 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 45 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 12 instances



Intersection of results for all words:

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html


In [209]:
searchword='bear eating trash'
multiple_search_webpages(searchword, vocab_trie, inverted_indices,filter_words)

input: bear eating trash 

'bear' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Brown_bear_ Wikipedia.html 324 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 262 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html 233 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 214 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 38 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 9 instances


'eating' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 7 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 6 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 6 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 5 instances

In [203]:
searchword='bear found in china'
multiple_search_webpages(searchword, vocab_trie, inverted_indices,filter_words)

input: bear found in china 

'bear' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Brown_bear_ Wikipedia.html 324 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 262 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html 233 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 214 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 38 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 9 instances


'found' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 25 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html 13 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 12 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 12 instan

In [219]:
searchword='which bear stays at north pole?'
multiple_search_webpages(searchword, vocab_trie, inverted_indices,filter_words)

input: which bear stays at north pole? 

'bear' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Brown_bear_ Wikipedia.html 324 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 262 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html 233 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 214 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 38 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 9 instances


'stays' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 1 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html 1 instances


'north' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 49 instances
C:/Users/kanad/Documents/C

## Textbox for search

In [221]:
search = input("Search: \t")
multiple_search_webpages(search, vocab_trie, inverted_indices,filter_words)

Search: 	bear in alaska
input: bear in alaska 

'bear' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Brown_bear_ Wikipedia.html 324 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 262 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html 233 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wikipedia.html 214 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Giant_panda _Wikipedia.html 38 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Red_panda_Wikipedia.html 9 instances


'alaska' found in these pages: 

C:/Users/kanad/Documents/CS 600/Project/HTML_files\Brown_bear_ Wikipedia.html 18 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\American_black_bear_Wikipedia.html 16 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Polar_bear_Wikipedia.html 12 instances
C:/Users/kanad/Documents/CS 600/Project/HTML_files\Bear_Wi